In [1]:
from dataclasses import dataclass
from functools import reduce
from pathlib import Path

data_file = Path("../Data/day5.txt").read_text()


EXAMPLE = """47|53
97|13
97|61
97|47
75|29
61|13
75|53
29|13
97|29
53|29
61|53
97|53
61|29
47|13
75|47
97|75
47|61
75|61
47|29
75|13
53|13

75,47,61,53,29
97,61,53,29,13
75,29,13
75,97,47,61,53
61,13,29
97,13,75,29,47"""


@dataclass
class PrinterPageOrderRule:
    X: int
    Y: int


@dataclass
class PrinterData:
    page_order_rules: list[PrinterPageOrderRule]
    updates: list[list[int]] | None


def prepare(input: str):
    def group_line(data: PrinterData, line: str):
        if line == "" and data.updates is None:
            data.updates = []

            return data

        if data.updates is not None:
            data.updates.append(list(map(int, line.split(","))))

            return data

        page_order_rule = line.split("|")

        data.page_order_rules.append(
            PrinterPageOrderRule(int(page_order_rule[0]), int(page_order_rule[1]))
        )

        return data

    return reduce(
        group_line, input.splitlines(), PrinterData(page_order_rules=[], updates=None)
    )


data = prepare(data_file)
example_data = prepare(EXAMPLE)

In [2]:
def map_rules(data: PrinterData):
    mapped_rules: dict[str, dict[str, list[PrinterPageOrderRule]]] = {"X": {}, "Y": {}}
    for rule in data.page_order_rules:
        try:
            mapped_rules["X"][str(rule.X)].append(rule)
        except KeyError:
            mapped_rules["X"][str(rule.X)] = [rule]

        try:
            mapped_rules["Y"][str(rule.Y)].append(rule)
        except KeyError:
            mapped_rules["Y"][str(rule.Y)] = [rule]

    return mapped_rules


def check_update_is_in_right_order(
    update: list[int], mapped_rules: dict[str, dict[str, list[PrinterPageOrderRule]]]
):
    for i, item in enumerate(update):
        for j in range(i + 1, len(update)):
            next_update = update[j]
            y_rules = mapped_rules["Y"].get(str(next_update))
            if y_rules is None:
                return False

            rule_found: PrinterPageOrderRule | None = None
            for rule in y_rules:
                if rule.X != item or rule.Y != next_update:
                    continue

                rule_found = rule

            if rule_found is None:
                return False

    return True


def part1(data: PrinterData):
    assert data.updates is not None

    mapped_rules = map_rules(data)
    valid_updates: list[int] = []
    for update in data.updates:
        update_is_in_right_order = check_update_is_in_right_order(
            update=update, mapped_rules=mapped_rules
        )
        if not update_is_in_right_order:
            continue

        valid_updates.append(update[len(update) // 2])

    return sum(valid_updates)


assert part1(example_data) == 143

result = part1(data)

print("result is", result)

result is 4135
